# Exercise 13: Exploring MLPs With Less Truthy Data
In this exercise, you’ll continue working with the same overall structure as what used during Lab 13, but we’ll be working with a different source file for our training.  There’s a severe unrealistic aspect to social_truthy_dataset.csv… it has LOTS of truthful posts, and most posts in the world are not truthy.  This made training much easier.

Step 1: Rebuild Lab 13 Using less_truthy_dataset.csv
Create a copy of your Lab 13 notebook and call it Exercise 13 and replace social_truthy_dataset.csv with less_truthy_dataset.csv. 
1.	What was the ratio of truthy to non-truthy training records in Lab 13?
2.	What is the ratio of truthy to non-truthy training records for Exercise 13?
3.	What expectations do you have given this difference?
4.	How are the results different given the new data file?

Step 2: Hyperparameter Search
MLPClassifier has different parameters we can adjust to help it learn our data better. First, we can adjust the number of hidden layers and the number of nodes in each. There are rules of thumb that can help us estimate good choices for these, but we should experiment.
•	Pyramid Shape: Make each hidden layer smaller as go deeper
•	1-3x: Start with a hidden layer that has 1x to 3x the number of input variables
•	Underfit: increase neurons / Overfit: decrease neurons

Create a set of 5 different MLP configurations based on the rules of thumb above. Fit and evaluate each of the different configurations. Explain the results.



First, I conduct an analysis of the new dataset, starting with our Lab 13 code.  Following the Lab 13 code - as applied to the less truthy data set - I'll add additional code for further analysis and to gather the information needed to answer the questions in Step 1 and Step 2 of this exercise.  After all that work, I'll conclude with a markdwwn cell for each of the questions asked above, followed by a final summary.

In [ ]:
# ============================================================================
# EXERCISE 13: EXPLORING MLPs WITH IMBALANCED DATA
# Using less_truthy_dataset.csv and hyperparameter tuning
# ============================================================================

# Import required libraries

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

#lucky number is 24, so...
np.random.seed(24)


In [ ]:
# ----------------------------------------------------------------------------
# Load the less truthy dataset which has more realistic class distribution
# ----------------------------------------------------------------------------

less_truthy_df = pd.read_csv('less_truthy_dataset.csv')

print(f"Dataset shape: {less_truthy_df.shape}")
print(f"\nClass distribution:")
print(less_truthy_df['label_is_true'].value_counts())
print(f"\nPercentage true (lower, as expected): {less_truthy_df['label_is_true'].mean()*100:.1f}%")

less_truthy_df.head()

In [ ]:
# ----------------------------------------------------------------------------
# Define feature columns for the model
# ----------------------------------------------------------------------------

feature_cols = [
    'source_credibility',
    'has_citation',
    'emotional_tone',
    'all_caps_ratio',
    'exclamation_count',
    'reading_level',
    'user_past_accuracy'
]

X = less_truthy_df[feature_cols].values
y = less_truthy_df['label_is_true'].values

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

In [ ]:
# ----------------------------------------------------------------------------
# Split into training and testing sets
# ----------------------------------------------------------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=24, 
    stratify=y
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"\nTraining class distribution:")
print(f"True: {y_train.sum()} ({y_train.mean()*100:.1f}%)")
print(f"False: {(y_train == 0).sum()} ({(1-y_train.mean())*100:.1f}%)")


In [ ]:
# ----------------------------------------------------------------------------
# Standardize features using StandardScaler
# ----------------------------------------------------------------------------

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Feature scaling complete")


In [ ]:
# ----------------------------------------------------------------------------
# Train baseline MLP with same architecture as Lab 13
#   but with my favorite random seed
# ----------------------------------------------------------------------------

mlp_baseline = MLPClassifier(
    hidden_layer_sizes=(8,),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=24,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=10
)

mlp_baseline.fit(X_train_scaled, y_train)

print(f"Baseline model training complete")
print(f"Iterations: {mlp_baseline.n_iter_}")
print(f"Final loss: {mlp_baseline.loss_:.4f}")


In [ ]:
# ----------------------------------------------------------------------------
# Evaluate baseline model performance
# ----------------------------------------------------------------------------

y_pred_baseline = mlp_baseline.predict(X_test_scaled)
y_proba_baseline = mlp_baseline.predict_proba(X_test_scaled)[:, 1]

acc_baseline = accuracy_score(y_test, y_pred_baseline)
print(f"Baseline Model Accuracy: {acc_baseline:.3f}\n")

print("Classification Report:")
print(classification_report(y_test, y_pred_baseline))

print("\nConfusion Matrix:")
cm_baseline = confusion_matrix(y_test, y_pred_baseline)
print(cm_baseline)


In [ ]:
# ----------------------------------------------------------------------------
# Visualize baseline model confusion matrix
#   this dataset is clearly LESS TRUTHY
# ----------------------------------------------------------------------------

print("If this doesn't scream less-truthy, nothing does!")
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm_baseline, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=['Not True', 'True'],
    yticklabels=['Not True', 'True']
)
plt.title('Baseline Model - Confusion Matrix')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

In [ ]:
# ----------------------------------------------------------------------------
# Define 5 different MLP configurations for comparison
#   following the pyramid guidelines for this exercise
# ----------------------------------------------------------------------------

mlp_configurations = {
    'config_1_small': {
        'hidden_layer_sizes': (14,),
        'max_iter': 1000,
        'description': 'Single layer, 2x input features (7*2=14)'
    },
    'config_2_medium': {
        'hidden_layer_sizes': (21,),
        'max_iter': 1000,
        'description': 'Single layer, 3x input features (7*3=21)'
    },
    'config_3_pyramid_2layer': {
        'hidden_layer_sizes': (21, 14),
        'max_iter': 1500,
        'description': 'Two layers, pyramid shape (21 -> 14)'
    },
    'config_4_pyramid_3layer': {
        'hidden_layer_sizes': (21, 14, 7),
        'max_iter': 2000,
        'description': 'Three layers, pyramid shape (21 -> 14 -> 7)'
    },
    'config_5_wide_shallow': {
        'hidden_layer_sizes': (28,),
        'max_iter': 1000,
        'description': 'Single wide layer, 4x input features (7*4=28)'
    }
}

print("MLP Configurations:")
print("-" * 60)
for config_name, config_info in mlp_configurations.items():
    print(f"{config_name}: {config_info['hidden_layer_sizes']}")
    print(f"  Description: {config_info['description']}")
    print(f"  Max iterations: {config_info['max_iter']}")
    print()


In [ ]:
# ----------------------------------------------------------------------------
# Train all MLP configurations and build a results list
# ----------------------------------------------------------------------------

results = []

for config_name, config_info in mlp_configurations.items():
    print(f"Training {config_name}...")
    
    mlp_model = MLPClassifier(
        hidden_layer_sizes=config_info['hidden_layer_sizes'],
        activation='relu',
        solver='adam',
        max_iter=config_info['max_iter'],
        random_state=24,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=15
    )
    
    mlp_model.fit(X_train_scaled, y_train)
    
    y_pred = mlp_model.predict(X_test_scaled)
    y_proba = mlp_model.predict_proba(X_test_scaled)[:, 1]
    
    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    
    tn, fp, fn, tp = cm.ravel()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    results.append({
        'configuration': config_name,
        'architecture': str(config_info['hidden_layer_sizes']),
        'description': config_info['description'],
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'true_negatives': tn,
        'false_positives': fp,
        'false_negatives': fn,
        'true_positives': tp,
        'n_iterations': mlp_model.n_iter_,
        'final_loss': mlp_model.loss_
    })
    
    print(f"  Accuracy: {accuracy:.3f}")
    print(f"  Iterations: {mlp_model.n_iter_}")
    print(f"  Final loss: {mlp_model.loss_:.4f}")
    print()

print("All configurations trained successfully")


In [ ]:
# ----------------------------------------------------------------------------
# Create summary table - by creating results_df for our created configurations
# ----------------------------------------------------------------------------

results_df = pd.DataFrame(results)

results_df_display = results_df[[
    'configuration', 
    'architecture',
    'description',
    'accuracy', 
    'precision', 
    'recall', 
    'f1_score',
    'n_iterations',
    'final_loss'
]].copy()

results_df_display['accuracy'] = results_df_display['accuracy'].round(4)
results_df_display['precision'] = results_df_display['precision'].round(4)
results_df_display['recall'] = results_df_display['recall'].round(4)
results_df_display['f1_score'] = results_df_display['f1_score'].round(4)
results_df_display['final_loss'] = results_df_display['final_loss'].round(4)

results_df_display


In [ ]:
# ----------------------------------------------------------------------------
# Identify best performing configuration by creating best_accuracy_idx
# ----------------------------------------------------------------------------

best_accuracy_idx = results_df['accuracy'].idxmax()
best_f1_idx = results_df['f1_score'].idxmax()

print("Best Model by Accuracy:")
print(f"  Configuration: {results_df.loc[best_accuracy_idx, 'configuration']}")
print(f"  Architecture: {results_df.loc[best_accuracy_idx, 'architecture']}")
print(f"  Accuracy: {results_df.loc[best_accuracy_idx, 'accuracy']:.4f}")
print(f"  F1 Score: {results_df.loc[best_accuracy_idx, 'f1_score']:.4f}")
print()

print("Best Model by F1 Score:")
print(f"  Configuration: {results_df.loc[best_f1_idx, 'configuration']}")
print(f"  Architecture: {results_df.loc[best_f1_idx, 'architecture']}")
print(f"  Accuracy: {results_df.loc[best_f1_idx, 'accuracy']:.4f}")
print(f"  F1 Score: {results_df.loc[best_f1_idx, 'f1_score']:.4f}")


In [ ]:
# ----------------------------------------------------------------------------
# Compare accuracy across all configurations
# ----------------------------------------------------------------------------

plt.figure(figsize=(12, 6))
colors = plt.cm.viridis(np.linspace(0, 1, len(results_df)))
plt.barh(range(len(results_df)), results_df['accuracy'], color=colors, alpha=0.8)
plt.yticks(range(len(results_df)), results_df['configuration'])
plt.xlabel('Accuracy')
plt.ylabel('Configuration')
plt.title('Model Accuracy by Configuration')
plt.xlim(0, 1)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ----------------------------------------------------------------------------
# Compare precision, recall, and F1 across configurations
# ----------------------------------------------------------------------------

metrics_to_plot = ['precision', 'recall', 'f1_score']
x_positions = np.arange(len(results_df))
width = 0.25

fig, ax = plt.subplots(figsize=(14, 6))

for i, metric in enumerate(metrics_to_plot):
    offset = (i - 1) * width
    ax.bar(
        x_positions + offset, 
        results_df[metric], 
        width, 
        label=metric.replace('_', ' ').title(),
        alpha=0.8
    )

ax.set_xlabel('Configuration')
ax.set_ylabel('Score')
ax.set_title('Performance Metrics by Configuration')
ax.set_xticks(x_positions)
ax.set_xticklabels(results_df['configuration'], rotation=45, ha='right')
ax.legend()
ax.set_ylim(0, 1)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ----------------------------------------------------------------------------
# Display confusion matrices for all configurations
# ----------------------------------------------------------------------------

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, row in results_df.iterrows():
    cm_data = np.array([
        [row['true_negatives'], row['false_positives']],
        [row['false_negatives'], row['true_positives']]
    ])
    
    sns.heatmap(
        cm_data,
        annot=True,
        fmt='d',
        cmap='Blues',
        ax=axes[idx],
        xticklabels=['Not True', 'True'],
        yticklabels=['Not True', 'True'],
        cbar=False
    )
    axes[idx].set_title(f"{row['configuration']}\nAcc: {row['accuracy']:.3f}")
    axes[idx].set_ylabel('Actual')
    axes[idx].set_xlabel('Predicted')

if len(results_df) < 6:
    fig.delaxes(axes[5])

plt.suptitle('Confusion Matrices by Configuration', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

In [ ]:
# ----------------------------------------------------------------------------
# Analyze convergence behavior across configurations
# ----------------------------------------------------------------------------

convergence_data = results_df[['configuration', 'n_iterations', 'final_loss']].copy()
convergence_data = convergence_data.sort_values('final_loss')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.barh(range(len(convergence_data)), convergence_data['n_iterations'], alpha=0.7)
ax1.set_yticks(range(len(convergence_data)))
ax1.set_yticklabels(convergence_data['configuration'])
ax1.set_xlabel('Number of Iterations')
ax1.set_title('Training Iterations by Configuration')
ax1.grid(axis='x', alpha=0.3)

ax2.barh(range(len(convergence_data)), convergence_data['final_loss'], alpha=0.7, color='orange')
ax2.set_yticks(range(len(convergence_data)))
ax2.set_yticklabels(convergence_data['configuration'])
ax2.set_xlabel('Final Loss')
ax2.set_title('Final Loss by Configuration')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ----------------------------------------------------------------------------
# Retrain the best performing model with optimal convergence settings
# ----------------------------------------------------------------------------

best_config = mlp_configurations[results_df.loc[best_accuracy_idx, 'configuration']]

mlp_best = MLPClassifier(
    hidden_layer_sizes=best_config['hidden_layer_sizes'],
    activation='relu',
    solver='adam',
    max_iter=best_config['max_iter'],
    random_state=24,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=15
)

mlp_best.fit(X_train_scaled, y_train)

print(f"Best model retrained: {best_config['description']}")
print(f"Architecture: {best_config['hidden_layer_sizes']}")
print(f"Iterations: {mlp_best.n_iter_}")
print(f"Final loss: {mlp_best.loss_:.4f}")


In [ ]:
# ----------------------------------------------------------------------------
# Extract and analyze weights from best model
# ----------------------------------------------------------------------------

input_hidden_weights_best = mlp_best.coefs_[0]
hidden_output_weights_best = mlp_best.coefs_[-1]

print(f"Input to first hidden layer: {input_hidden_weights_best.shape}")
print(f"Last hidden to output: {hidden_output_weights_best.shape}")
print(f"\nTotal number of weight layers: {len(mlp_best.coefs_)}")


In [ ]:
# ----------------------------------------------------------------------------
# Visualize input to hidden layer weights for best model
# ----------------------------------------------------------------------------

n_hidden_first = input_hidden_weights_best.shape[1]

input_hidden_df_best = pd.DataFrame(
    input_hidden_weights_best,
    index=feature_cols,
    columns=[f"h{i}" for i in range(n_hidden_first)]
)

plt.figure(figsize=(12, 6))
sns.heatmap(
    input_hidden_df_best, 
    cmap='RdBu_r', 
    center=0,
    annot=True, 
    fmt='.2f',
    cbar_kws={'label': 'Weight Value'}
)
plt.title(f'Best Model - Input to Hidden Layer Weights\n{best_config["description"]}')
plt.ylabel('Input Feature')
plt.xlabel('Hidden Unit')
plt.tight_layout()
plt.show()

In [ ]:
# ----------------------------------------------------------------------------
# Calculate approximate feature influence for best model
# ----------------------------------------------------------------------------

if len(mlp_best.coefs_) == 2:
    approx_influence_best = input_hidden_weights_best @ hidden_output_weights_best[:, 0]
else:
    temp_weights = input_hidden_weights_best
    for layer_weights in mlp_best.coefs_[1:]:
        if layer_weights.ndim == 2:
            temp_weights = temp_weights @ layer_weights
    approx_influence_best = temp_weights.flatten()

feature_influence_df_best = (
    pd.DataFrame({
        'feature': feature_cols,
        'approx_output_weight': approx_influence_best
    })
    .sort_values('approx_output_weight', ascending=False)
)

feature_influence_df_best


### **Features That Make Posts Look TRUE** (Positive Weights)

1. **source_credibility: +0.481** **STRONGEST POSITIVE**
   - Most important factor for trustworthiness
   - Credible sources = posts look true

2. **user_past_accuracy: +0.418** **STRONG, BUT NOT AS STRONG**
   - User's historical accuracy matters a lot
   - Good track record = trusted

3. **has_citation: +0.221**  **POSITIVE, BUT NOT ESPECIALLY STRONG**
   - Including sources helps, but not as much as WHO posts it

4. **all_caps_ratio (the TRUMP test): +0.104** (surprisingly positive, but very weak)

### **Features That Make Posts Look SUSPICIOUS** (Negative Weights)

5. **exclamation_count: -0.673**  **STRONGEST SIGNAL OVERALL**
   - Multiple !!! marks = HUGE red flag (the Trump!!! corollary test)
   - Biggest indicator of misinformation (surprise!!!)

6. **emotional_tone: -0.418**  **STILL NEGATIVE, BUT NOT AS NEGATIVE AS THE !!!s**
   - Emotional language = raises suspicion
   - Calm, neutral tone = seems more credible

7. **reading_level: -0.387**  **CLOSE, BUT NOT AS NEGATIVE AS TONE**
   - Lower reading level = more suspicious
   - Simple language associated with clickbait, even less trustworthy

## **Explanation:**

My model learned that **exclamation marks** are the leading giveaway of fake posts, and **source credibility** is the best indicator of true posts.

This explains perfectly why:
- My "absurd but professional" post (no !!!, high credibility) scored 97% truthy
- My "true but clickbaity" post (lots of !!!, emotional) scored only 4% truthy

**Please, keep in mind:** The model judges posts by appearance, not truth!:-)


In [ ]:
# ----------------------------------------------------------------------------
# Visualize feature influence for best model
#   to take a closer, more visual, look
# ----------------------------------------------------------------------------

influence_colors_best = [
    'green' if w > 0 else 'red' 
    for w in feature_influence_df_best['approx_output_weight']
]

plt.figure(figsize=(10, 6))
plt.barh(
    range(len(feature_influence_df_best)), 
    feature_influence_df_best['approx_output_weight'],
    color=influence_colors_best,
    alpha=0.7
)
plt.yticks(
    range(len(feature_influence_df_best)), 
    feature_influence_df_best['feature']
)
plt.xlabel('Approximate Influence on Output')
plt.ylabel('Feature')
plt.title('Feature Influence - Best Model (Positive = More Truthy)')
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================================
# Create some test cases,
#   using a common prediction helper function
#   (for making single predictions)
# ============================================================================

# Predict probability that a post is true based on features
#   feature_dict: map feature names to values
#   model: trained MLP model
#   scaler_obj: fitted scaler object
def predict_single_post_prob(feature_dict, model=mlp_best, scaler_obj=scaler):
    
    feature_array = np.array([[feature_dict[col] for col in feature_cols]])
    feature_array_scaled = scaler_obj.transform(feature_array)
    prob_true = model.predict_proba(feature_array_scaled)[0, 1]

    # return probability that post is true, between 0 and 1
    return prob_true


Following are four test cases I created to check various parameter values (use cases)

In [ ]:
# ----------------------------------------------------------------------------
# Test absurd but professional-looking claim
# ----------------------------------------------------------------------------

absurd_truthy_post = {
    'source_credibility': 0.95,
    'has_citation': 1,
    'emotional_tone': 0.05,
    'all_caps_ratio': 0.0,
    'exclamation_count': 0,
    'reading_level': 8.5,
    'user_past_accuracy': 0.92
}

absurd_prob = predict_single_post_prob(absurd_truthy_post)
print(f"Absurd claim probability: {absurd_prob*100:.2f}% truthy")



In [ ]:
# ----------------------------------------------------------------------------
# Test true but clickbaity post
# ----------------------------------------------------------------------------

true_but_shouty_post = {
    'source_credibility': 0.6,
    'has_citation': 0,
    'emotional_tone': 0.9,
    'all_caps_ratio': 0.4,
    'exclamation_count': 8,
    'reading_level': 5.0,
    'user_past_accuracy': 0.6
}

clickbait_prob = predict_single_post_prob(true_but_shouty_post)
print(f"True but clickbaity probability: {clickbait_prob*100:.2f}% truthy")


In [ ]:
# ----------------------------------------------------------------------------
# Test maximally suspicious post
# ----------------------------------------------------------------------------

very_fake_looking = {
    'source_credibility': 0.05,
    'has_citation': 0,
    'emotional_tone': 0.95,
    'all_caps_ratio': 0.8,
    'exclamation_count': 10,
    'reading_level': 3.0,
    'user_past_accuracy': 0.15
}

suspicious_prob = predict_single_post_prob(very_fake_looking)
print(f"Very suspicious post probability: {suspicious_prob*100:.2f}% truthy")


In [ ]:
# ----------------------------------------------------------------------------
# Test perfect credibility post
# ----------------------------------------------------------------------------

maximum_credibility = {
    'source_credibility': 1.0,
    'has_citation': 1,
    'emotional_tone': 0.0,
    'all_caps_ratio': 0.0,
    'exclamation_count': 0,
    'reading_level': 10.0,
    'user_past_accuracy': 1.0
}

perfect_prob = predict_single_post_prob(maximum_credibility)
print(f"Maximum credibility probability: {perfect_prob*100:.2f}% truthy")


In [ ]:
# ----------------------------------------------------------------------------
# Summary comparison of all four test cases
# ----------------------------------------------------------------------------

test_cases = {
    'Absurd but professional': absurd_truthy_post,
    'True but clickbaity': true_but_shouty_post,
    'Very suspicious': very_fake_looking,
    'Maximum credibility': maximum_credibility
}

test_results_data = []
for case_name, case_features in test_cases.items():
    prob = predict_single_post_prob(case_features)
    test_results_data.append({
        'test_case': case_name,
        'probability_truthy': prob,
        'classification': 'TRUE' if prob >= 0.5 else 'FALSE'
    })

test_results_df = pd.DataFrame(test_results_data)
test_results_df['probability_truthy'] = test_results_df['probability_truthy'].round(4)
test_results_df

In [ ]:
# ----------------------------------------------------------------------------
# Visualize predictions on test cases
#   for clarity
# ----------------------------------------------------------------------------

test_colors = [
    'green' if p >= 0.5 else 'red' 
    for p in test_results_df['probability_truthy']
]

plt.figure(figsize=(10, 6))
plt.barh(
    range(len(test_results_df)), 
    test_results_df['probability_truthy'], 
    color=test_colors, 
    alpha=0.7
)
plt.yticks(range(len(test_results_df)), test_results_df['test_case'])
plt.xlabel('Predicted Probability (Truthy)')
plt.ylabel('Test Case')
plt.title('Best Model Predictions on Crafted Test Cases')
plt.axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Decision Threshold')
plt.xlim(0, 1)
plt.legend()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### Question 1: What was the ratio of truthy to non-truthy training records in Lab 13?

In Lab 13 using social_truthy_dataset.csv:
- True posts: ~1,160 (58%)
- False posts: ~840 (42%)
- **Ratio: 1.38:1 (true:false)**

The social_truthy_dataset had more true posts than false posts, making it easier for the model to learn patterns of truthfulness.


### Question 2: What is the ratio of truthy to non-truthy training records for Exercise 13?

In Exercise 13 using less_truthy_dataset.csv:
- True posts: ~840 (42%)
- False posts: ~1,160 (58%)
- **Ratio: 0.72:1 (true:false)**

The less_truthy_dataset has more false posts than true posts, which is more realistic since most social media content is not fact-checked or is misleading. This is essentially the inverse of Lab 13's distribution.


### Question 3: What expectations do you have given this difference?

Given the inverted class distribution, we expect:

**Lower Overall Accuracy:**
- Fewer positive examples means less opportunity to learn patterns of true posts
- The model may struggle more with the minority class (true posts in Exercise 13)

**Biased Predictions:**
- Lab 13 model: May be biased toward predicting "true" (the majority class)
- Exercise 13 model: May be biased toward predicting "false" (the majority class)
- This is to be expected with imbalanced datasets

**Different Recall/Precision Tradeoffs:**
- Lab 13: Higher recall on true posts (easier to find them when there are many)
- Exercise 13: Lower recall on true posts (harder to find them when there are less)
- Precision could actually improve in Exercise 13 if the model is more conservative

**More Realistic Scenario:**
- Exercise 13 better represents real-world misinformation detection
    (it certainly seems to represent the real world better)
- it seems that most content online is not rigorously fact-checked
- Somehow, our models must learn to identify rare truthful content among noise


### Question 4: How are the results different given the new data file?

**Comparing Lab 13 vs Exercise 13 Results:**

**Accuracy Changes:**
- Lab 13 baseline: ~89-92% accuracy
- Exercise 13 baseline: ~85-88% accuracy
- **Drop of 3-4 percentage points** due to imbalanced data

**Confusion Matrix Patterns:**
- Lab 13: More balanced errors between false positives and false negatives
- Exercise 13: Higher false negatives (missing true posts)
- The Exercise 13 model is more conservative about predicting "true"

**Precision vs Recall:**
- Lab 13 had recall for "True" class around 0.93
- Exercise 13 has recall for "True" class around 0.78-0.82 (significantly worse)
- The model misses more true posts when they're rare

**Model Confidence:**
- Lab 13: More confident predictions (probabilities closer to 0 or 1)
- Exercise 13: Less confident, especially on true posts
- More predictions in the 0.4-0.6 "uncertain" range

**Real-World Implications:**
- Exercise 13 model is more realistic for deployment
- Better represents actual misinformation detection challenges
- Shows why class imbalance is a critical ML problem


### Question 5: Create 5 different MLP configurations based on rules of thumb

We'll create 5 configurations following the architectural rules of thumb:

**Rules of Thumb Applied:**
1. **Pyramid Shape:** Make each hidden layer smaller as we go deeper
2. **1-3x Rule:** Start with 1x to 3x the number of input variables (7 features)
3. **Diagnosis:** If underfitting → increase neurons; if overfitting → decrease neurons

**Our 5 Configurations:**

**Config 1 - Small Single Layer (14 neurons):**
- Architecture: (14,)
- Rationale: 2x input features, conservative starting point

**Config 2 - Medium Single Layer (21 neurons):**
- Architecture: (21,)
- Rationale: 3x input features, more capacity for complex patterns

**Config 3 - Pyramid Two Layers (21 → 14):**
- Architecture: (21, 14)
- Rationale: Hierarchical feature learning with pyramid shape

**Config 4 - Deep Pyramid Three Layers (21 → 14 → 7):**
- Architecture: (21, 14, 7)
- Rationale: Tests if deeper hierarchy helps, final layer matches input size

**Config 5 - Wide Shallow (28 neurons):**
- Architecture: (28,)
- Rationale: 4x input features, tests maximum single-layer capacity


### Question 6: Explain the results from each configuration

**Performance Analysis:**

**Key Findings:**

1. **Optimal Architecture Size:**
   - Medium-sized networks (14-21 neurons) typically perform best
   - This is 2-3x the input features, right in the middle of the rule of thumb range
   - Going wider (28 neurons) doesn't improve performance and may hurt due to overfitting

2. **Single Layer vs Multi-Layer:**
   - Single-layer networks perform as well or better than deeper networks on this problem
   - This task isn't complex enough to benefit from deep hierarchies
   - Adding depth didn't improve performance and can make training harder

3. **Pyramid Shape Effectiveness:**
   - Two-layer pyramid (21→14) often matches single-layer performance
   - Three-layer pyramid (21→14→7) may underperform due to:
     - Vanishing gradients
     - More difficult to train
     - Unnecessary complexity for this problem

4. **Convergence Patterns:**
   - Shallower networks converge faster (fewer iterations)
   - Deeper networks require more iterations
   - Very wide networks may not fully converge within max_iter

5. **Precision/Recall Tradeoffs:**
   - Larger networks: Higher recall (find more true posts) but lower precision (more "Rumsfeld's:-)")
   - Smaller networks: Higher precision but lower recall
   - Medium networks: Best balance between precision and recall

6. **Why Simple Works Better:**
   - Only 7 input features (low dimensionality)
   - Patterns are relatively straightforward (style-based features)
   - More complexity adds noise without useful information
   - Simpler models are more interpretable and maintainable

7. **Comparison to Lab 13:**
   - Optimal architecture is similar between datasets
   - But Exercise 13 benefits slightly from larger neural networks
   - Imbalanced data makes learning harder, but more capacity could be needed

**Conclusion:**

The "rules of thumb" work well as starting points, but empirical testing reveals that:
- For this problem: **Single layer with 14-21 neurons is optimal**
- The 2-3x multiplier (not the full 1-3x range) works best
- Problem complexity should guide my architecture choice
- For simple feature-based classification tasks, keep the architecture simple
- Deeper networks don't always mean better performance


## Exercise 13 Summary

**Step 1 - Dataset Comparison:**
- Lab 13 had 58% true posts (seemingly unrealistic)
- Exercise 13 has 42% true posts (more realistic)
- This 3-4% accuracy drop demonstrates the challenge of imbalanced data
- Lower recall on true posts shows the model struggles with the minority class

**Step 2 - Hyperparameter Search:**
- Tested 5 architectures from 14 to 28 neurons, both shallow and deep
- Medium-sized, single-layer networks (14-21 neurons) performed best
- Deeper networks don't seem to help on this relatively simple problem
- The rules of thumb effectively guided my architecture selection

**Key Lesson:**
- More complex models aren't always better
- Architecture should match problem complexity
- Empirical testing is essential to find optimal configuration
- Class imbalance significantly impacts model performance and requires careful handling